In [27]:
import warnings; 
warnings.simplefilter('ignore')

In [28]:
import pandas as pd
from fbprophet import Prophet

In [29]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [45]:
M_df = pd.read_csv('Master_data_ML.csv')
M_df.dtypes

county                          object
year                             int64
totalpopulation                  int64
average_yearly_wage              int64
lowtiermedianhomevalue         float64
hightiermedianhomevalue        float64
singlefamilymedianhomevalue    float64
value_inches                   float64
murder                         float64
rape                           float64
robbery                        float64
aggravated_assault             float64
burglary                       float64
larceny                        float64
motor_vehicle_theft            float64
index_per_100k                 float64
employment                     float64
unemployment                     int64
labor_force                      int64
unemployment_rate              float64
county_code                      int64
SFMHVrange                      object
SFMHVclass                       int64
dtype: object

In [52]:
M_df['year'] = M_df['year'].map(str) +'-01-01'
M_df['year'] = pd.to_datetime(M_df['year'])
M_df['year'] = M_df['year'].dt.date
M_df

,county,year,totalpopulation,average_yearly_wage,lowtiermedianhomevalue,hightiermedianhomevalue,singlefamilymedianhomevalue,value_inches,murder,rape,robbery,aggravated_assault,burglary,larceny,motor_vehicle_theft,index_per_100k,employment,unemployment,labor_force,unemployment_rate,county_code,SFMHVrange,SFMHVclass
0,Alachua,2001-01-01,222935,26917,72233.0,198921.0,121316.0,45.63,8.0,176.0,453.0,1240.0,2918.0,7736.0,1031.0,6083.0,114284.0,4263,118547,0.036,0,"(120000, 130000]",8
1,Alachua,2002-01-01,228607,27648,76757.0,210486.0,129499.0,58.40,7.0,189.0,335.0,1264.0,2768.0,7429.0,895.0,5637.0,113387.0,4776,118163,0.040,0,"(120000, 130000]",8
2,Alachua,2003-01-01,231296,28867,83740.0,228059.0,141447.0,53.70,7.0,160.0,304.0,1413.0,2542.0,6782.0,877.0,5225.0,114641.0,4446,119087,0.037,0,"(140000, 150000]",10
3,Alachua,2004-01-01,236174,30939,95977.0,260733.0,162329.0,64.46,8.0,194.0,310.0,1325.0,2533.0,6064.0,744.0,4733.0,115973.0,4283,120256,0.036,0,"(160000, 170000]",12
4,Alachua,2005-01-01,240764,33145,111818.0,301074.0,187998.0,54.20,7.0,233.0,291.0,1511.0,2442.0,6448.0,749.0,4852.0,119924.0,3626,123550,0.029,0,"(180000, 190000]",14
5,Alachua,2006-01-01,243779,34748,130667.0,345131.0,217399.0,37.30,13.0,239.0,382.0,1558.0,2779.0,7530.0,794.0,5454.0,122471.0,3372,125843,0.027,0,"(210000, 220000]",17
6,Alachua,2007-01-01,247561,36456,135485.0,354738.0,224103.0,44.10,7.0,197.0,409.0,1574.0,2920.0,7070.0,768.0,5229.0,125102.0,3908,129010,0.030,0,"(220000, 230000]",18
7,Alachua,2008-01-01,252388,37469,125463.0,325363.0,206437.0,45.17,4.0,173.0,353.0,1380.0,2747.0,7179.0,657.0,4950.0,125425.0,5821,131246,0.044,0,"(200000, 210000]",16
8,Alachua,2009-01-01,256232,39080,110449.0,288381.0,183571.0,53.10,5.0,191.0,374.0,1414.0,2858.0,7253.0,686.0,4988.0,120579.0,9236,129815,0.071,0,"(180000, 190000]",14
9,Alachua,2010-01-01,247336,39688,102223.0,269990.0,172110.0,45.00,12.0,154.0,298.0,1093.0,2297.0,6387.0,493.0,4198.0,117504.0,9880,127384,0.078,0,"(170000, 180000]",13


In [ ]:
for county in M_df.county:
    